In [1]:
###
### PYTHON PACKAGES
###
import pandas as pd
import re
import emoji
from ftfy import fix_text 
import unidecode
from langdetect import detect, detect_langs
import spacy
from spacy import displacy

###
### USER-DEVELOPED IMPORTS
###
# from project import *
from capture_group import *

### New Idea

1) loop through cleaned dataframe to find tweets with "win," "present," "nominate," "best" (maybe), "award" (maybe)
2) with a match, get any names

In [2]:
cleaned_data = pd.read_csv('text_cleaned.csv')['text']
win_data = pd.read_csv('wins.csv')['text']
spacy_data = pd.read_csv('spacy_info.csv')

In [3]:
spacy_size = 1500

In [138]:
# win_data.head(50).apply(lambda x: find_award_winner(x))x
find_award_winner("I hope Yash or Sorie wins best student ")

'Yash | best student | nominee'

In [171]:
def nominee_search(text):
    doc = nlp(text)
    
    entities = extract_entities(doc)
    
    gen_patterns = r'(is nominated for|is a nominee for)'
    if re.search(gen_patterns, text, re.IGNORECASE):
        for ent in entities:
            if ent[1] == "PERSON":
                alleged_winner = ent[0]
        
        award = extract_award_name_after_best(doc)
        
        if award != None:
            return alleged_winner, award
        
        word_list = ["award", "prize", "honor", "medal", "trophy"]
        pattern = r"(\w+)\s+" + gen_patterns + r"(.*?\b(?:" + "|".join(re.escape(word) for word in word_list) + r")\b)"
        match = re.search(pattern, text, re.IGNORECASE)
        
        return alleged_winner, match.group(3)

In [174]:
nominee_search("Yash is a nominee for best student")

('Yash', 'best student')

### Notes from office hours

There may be accounts (i.e. newspapers) that HAVE to post who the nominees are, look through data for that
Can use ordering of events (i.e. award A goes Speaker -> Nominees -> Winner)
Start by looking at data that has information you know you need
Try inputting JSON into ChatGPT and ask it to return tweets where nominees for awards are mentioned
Somebody mentioned frequently near mention of an award winner

For Spacy, find nsubj+, check IMDB to see if it's actually a movie or actor/actress (if time)

Based on award name, can tell if you need a movie or actor / actress / song

See what Spacy outputs for "movie wins x award"


In [38]:
from itertools import islice

account_rts = {}

for text in cleaned_data:
    tweet = text.split(' ')
    if tweet[0] == 'rt':
        if tweet[1] in account_rts:
            account_rts[tweet[1]] += 1
        else:
            account_rts.update({tweet[1]:1})

account_dicts = dict(sorted(account_rts.items(), key=lambda item: item[1], reverse=True))

top_accounts = list(islice(account_dicts, 10))
top_accounts

print(top_accounts)

['@goldenglobes', '@eonline', '@perezhilton', '@theellenshow', '@emwatson', '@vanityfair', '@nbcsnl', '@cnnshowbiz', '@ciudadbizarra', '@ew']


In [33]:
retweet_pattern = r're\s' + '|'.join(top_accounts)
match = re.search(retweet_pattern, text, re.IGNORECASE)

In [93]:
def account_rt(text):
    doc = nlp(text)
    print(text, " --- 1")
    
    pattern = r're \s' + '|'.join(top_accounts)
    
    if re.search(pattern, text, re.IGNORECASE):     
        print(text, " --- 2")   
        return find_award_winner(text)

In [94]:
text = "rt @goldenglobes best screenplay quentin tarantino django unchained"

In [95]:
account_rt(text)

rt @goldenglobes best screenplay quentin tarantino django unchained  --- 1


In [96]:
df = pd.read_json('gg2013.json')['text']

In [97]:
df[102541]

'RT @goldenglobes: Best Screenplay - Quentin Tarantino - Django Unchained - #GoldenGlobes'

In [98]:
nominee_data = cleaned_data[cleaned_data.apply(lambda x: re.search(r'\b(nominee|nominate|nominated)\b', x) != None)]

nominee_data.to_csv("nominees.csv")

In [99]:
nominee_data

49        rt @lokipage times we wish good luck tonight t...
61        rt @twittermovies nominee @officialadele on th...
154       ive seen a grand total of one nominated film s...
338       rt @zap2itrick can kerry washington be nominat...
505       rt @twittermovies nominee @officialadele on th...
                                ...                        
173040    rt @jessicacapshaw thank u all for your play b...
173059    the were amazing im so happy argo won but i th...
173107    rt @oskrnyc "7 cajas" should have been nominat...
173130    "@itsjerrah i should be nominated as best acto...
173203    rt @mapeel yes should have been nominated for ...
Name: text, Length: 1037, dtype: object